<a href="https://colab.research.google.com/github/shashi3876/kaggle/blob/main/BeyondVisibleSpectrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [ ]:
folder = 'ot/ot/'
npy_data = {}

# Loop through all .npy files
for filepath in glob.glob(os.path.join(folder, '*.npy')):
    key = os.path.splitext(os.path.basename(filepath))[0]
    try:
        npy_data[key] = np.load(filepath)
    except Exception as e:
        print(f"Skipping {key} due to error: {e}")

# Optional: preview keys
print(npy_data.keys())


In [ ]:
expected_shape = (128, 128, 125)

for key, arr in npy_data.items():
    if arr.shape != expected_shape:
        print(f"File: {key}.npy --> Shape: {arr.shape}")